# Challenge Valeo

https://challengedata.ens.fr/participants/challenges/36/

In [81]:
# Import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer


In [82]:
# Give some style to my texts

class style:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Importer les données

In [83]:
data_x = pd.read_csv('x_train.csv')
data_y = pd.read_csv('y_train.csv')

In [84]:
data = data_x.join(data_y.set_index('PROC_TRACEINFO'), on='PROC_TRACEINFO')

## Analyse des données

In [85]:
# First look on features

print(style.BOLD + "Features description: \n" + style.END, data.describe(include='all'))
print()
print(style.BOLD + "initial dataset shape: \n" + style.END, data.shape)
print()
print(style.BOLD + "Features types: \n" + style.END, data.dtypes)
print()
print(style.BOLD + "Missing values: \n" + style.END, data.isnull().any())

Features description: 
                     PROC_TRACEINFO  OP070_V_1_angle_value  \
count                        34515           34515.000000   
unique                       34515                    NaN   
top     I-B-XA1207672-190530-03680                    NaN   
freq                             1                    NaN   
mean                           NaN             159.906922   
std                            NaN              15.662650   
min                            NaN             101.800000   
25%                            NaN             148.700000   
50%                            NaN             158.000000   
75%                            NaN             169.300000   
max                            NaN             198.300000   

        OP090_SnapRingPeakForce_value  OP070_V_2_angle_value  \
count                    34515.000000           34515.000000   
unique                            NaN                    NaN   
top                               NaN              

- 34515 lignes pour 15 colonnes (dont 1 pour target)
- target : 'Binar OP130_Resultat_Global_v'
- valeurs manquantes dans OP100_Capuchon_insertion_mesure (18627 soit 53.96784006953499%)
- normaliser les features
- aucune feature à catégoriser
- Y n'est pas à normaliser ou catégoriser


## Data Viz

In [86]:
# plus tard

## Preprocessing

In [87]:
# Separate target variable Y from features X
print("Separating labels from features...")
features_list = list(data.columns[1:-1])
target_variable = 'Binar OP130_Resultat_Global_v'

X = data.loc[:,features_list]
Y = data.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
0    0
1    0
2    0
3    0
4    0
Name: Binar OP130_Resultat_Global_v, dtype: int64

X :
   OP070_V_1_angle_value  OP090_SnapRingPeakForce_value  \
0                  180.4                         190.51   
1                  138.7                         147.70   
2                  180.9                         150.87   
3                  173.5                         159.56   
4                  174.5                         172.29   

   OP070_V_2_angle_value  OP120_Rodage_I_mesure_value  \
0                  173.1                       113.64   
1                  163.5                       109.77   
2                  181.2                       109.79   
3                  151.8                       113.25   
4                  177.5                       112.88   

   OP090_SnapRingFinalStroke_value  OP110_Vissage_M8_torque_value  \
0                            12.04                          12.16   
1                       

In [88]:
idx = 0
numeric_features = []
numeric_indices = []
categorical_features = []
categorical_indices = []
for i,t in X.dtypes.iteritems():
  if ('float' in str(t)) or ('int' in str(t)) :
    numeric_features.append(i)
    numeric_indices.append(idx)
  else :
    categorical_features.append(i)
    categorical_indices.append(idx)

  idx = idx + 1

print('Found numeric features ', numeric_features,' at positions ', numeric_indices)
print('Found categorical features ', categorical_features,' at positions ', categorical_indices)

Found numeric features  ['OP070_V_1_angle_value', 'OP090_SnapRingPeakForce_value', 'OP070_V_2_angle_value', 'OP120_Rodage_I_mesure_value', 'OP090_SnapRingFinalStroke_value', 'OP110_Vissage_M8_torque_value', 'OP100_Capuchon_insertion_mesure', 'OP120_Rodage_U_mesure_value', 'OP070_V_1_torque_value', 'OP090_StartLinePeakForce_value', 'OP110_Vissage_M8_angle_value', 'OP090_SnapRingMidPointForce_val', 'OP070_V_2_torque_value']  at positions  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Found categorical features  []  at positions  []


In [89]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, 
                                                    random_state=42, stratify=Y)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [90]:
# Convert pandas DataFrames to numpy arrays before using scikit-learn
print("Convert pandas DataFrames to numpy arrays...")
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.tolist()
Y_test = Y_test.tolist()
print("...Done")

Convert pandas DataFrames to numpy arrays...
...Done


## Training pipeline

In [91]:
# Missing values
print("Imputing missing values...")
print(X_train[0:5,:])
print()
imputer = SimpleImputer(strategy="mean")
X_train[:,numeric_indices] = imputer.fit_transform(X_train[:,numeric_indices])
print("...Fini!")


Imputing missing values...
[[172.3  143.24 166.2  119.85  11.9   12.4     nan  11.97   6.61  23.07
   10.6   95.5    6.61]
 [154.1  153.71 169.7  109.63  12.09  12.35   0.42  11.97   6.4   23.92
   11.6  104.11   6.41]
 [149.3  145.82 158.9  108.89  12.12  12.25   0.39  11.97   6.4   18.51
   14.3   86.8    6.41]
 [197.9  157.39 187.6  111.65  11.7   12.4     nan  11.97   6.34  23.14
    9.2   95.3    6.6 ]
 [152.1  176.02 148.3  116.29  11.87  12.15    nan  11.97   6.6   22.35
   26.4   95.16   6.61]]

...Fini!


In [92]:
# Encoding categorical features and standardizing numerical features
print("Encoding categorical features and standardizing numerical features...")
print()
print(X_train[0:5,:])

# Normalization
numeric_transformer = StandardScaler()

# OHE / dummyfication
categorical_transformer = OneHotEncoder(drop='first')

featureencoder = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_indices),    
        ('num', numeric_transformer, numeric_indices)
        ]
    )

X_train = featureencoder.fit_transform(X_train)
print("...Done")


Encoding categorical features and standardizing numerical features...

[[172.3        143.24       166.2        119.85        11.9
   12.4          0.38817523  11.97         6.61        23.07
   10.6         95.5          6.61      ]
 [154.1        153.71       169.7        109.63        12.09
   12.35         0.42        11.97         6.4         23.92
   11.6        104.11         6.41      ]
 [149.3        145.82       158.9        108.89        12.12
   12.25         0.39        11.97         6.4         18.51
   14.3         86.8          6.41      ]
 [197.9        157.39       187.6        111.65        11.7
   12.4          0.38817523  11.97         6.34        23.14
    9.2         95.3          6.6       ]
 [152.1        176.02       148.3        116.29        11.87
   12.15         0.38817523  11.97         6.6         22.35
   26.4         95.16         6.61      ]]
...Done


In [93]:
'''
# Encoding labels
print("Encoding labels...")
print()
labelencoder = LabelEncoder()
Y_train = labelencoder.fit_transform(Y_train)
print("...Done.")
'''

'\n# Encoding labels\nprint("Encoding labels...")\nprint()\nlabelencoder = LabelEncoder()\nY_train = labelencoder.fit_transform(Y_train)\nprint("...Done.")\n'

## Test pipeline

In [94]:
# Missing values
print("Imputing missing values...")
print(X_test[0:5,:])
print()

X_test[:,numeric_indices] = imputer.transform(X_test[:,numeric_indices])
print("...Done.")
print(X_test[0:5,:]) 
print() 

Imputing missing values...
[[154.3  180.54 135.3  112.83  12.06  12.18    nan  11.97   6.62  23.13
   21.8  105.86   6.62]
 [162.9  155.39 152.3  113.89  12.05  12.2    0.41  11.97   6.42  21.97
   21.5  102.77   6.41]
 [161.2  139.4  139.   115.66  11.84  12.19   0.34  11.97   6.6   23.32
   25.5   93.51   6.63]
 [165.5  139.68 138.8  114.95  12.1   12.24   0.41  11.97   6.42  20.62
   14.3   95.94   6.41]
 [144.5  146.23 169.2  112.15  11.84  12.36    nan  11.97   6.62  27.94
   15.7  105.17   6.61]]

...Done.
[[154.3        180.54       135.3        112.83        12.06
   12.18         0.38817523  11.97         6.62        23.13
   21.8        105.86         6.62      ]
 [162.9        155.39       152.3        113.89        12.05
   12.2          0.41        11.97         6.42        21.97
   21.5        102.77         6.41      ]
 [161.2        139.4        139.         115.66        11.84
   12.19         0.34        11.97         6.6         23.32
   25.5         93.51         6.

In [95]:
# Encoding categorical features and standardizing numerical features
print("Encoding categorical features and standardizing numerical features...")
X_test = featureencoder.transform(X_test)
print("...Done")


Encoding categorical features and standardizing numerical features...
...Done


In [96]:
'''
# Encoding labels
print("Encoding labels...")

Y_test = labelencoder.transform(Y_test)
print("...Done.")
'''

'\n# Encoding labels\nprint("Encoding labels...")\n\nY_test = labelencoder.transform(Y_test)\nprint("...Done.")\n'

In [97]:
# A partir d'ici je commence les modèles